In [22]:
from torchvision import transforms
from torch import optim
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy
import sys
import os
import matplotlib.pyplot
import random

# Image processing
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize(mean=(0.5,), std=(0.5,))])

batch_size = 100

# MNIST dataset
train_mnist = torchvision.datasets.MNIST(root='./data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [23]:
for step, (x, y) in enumerate(train_loader):
    print(step)
    print(x.shape)
    print(y.shape)
    print(y)
    break

0
torch.Size([100, 1, 28, 28])
torch.Size([100])
tensor([0, 8, 9, 9, 9, 5, 0, 7, 5, 4, 2, 2, 8, 0, 5, 8, 7, 0, 2, 2, 3, 6, 6, 2,
        1, 8, 6, 6, 5, 9, 8, 0, 6, 4, 4, 1, 9, 4, 5, 9, 5, 3, 1, 8, 9, 9, 6, 6,
        5, 0, 3, 5, 8, 1, 0, 1, 1, 1, 0, 7, 8, 5, 4, 1, 3, 4, 4, 8, 2, 1, 2, 6,
        6, 7, 2, 9, 1, 9, 8, 4, 8, 7, 4, 2, 0, 6, 3, 2, 0, 1, 2, 7, 4, 2, 8, 0,
        9, 6, 0, 4])


In [24]:
class Gen_model(nn.Module):
    def __init__(self):
        super(Gen_model, self).__init__()
        self.lin1 = nn.Linear(10*10,20*20)
        self.lin2 = nn.Linear(20*20,24*24)
        self.lin3 = nn.Linear(24*24,28*28)
        
    def forward(self,x):
        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        x = F.relu(x)
        x = self.lin3(x)
        return x
    
class Discr_model(nn.Module):
    def __init__(self):
        super(Discr_model, self).__init__()
        self.conv1 = nn.Conv2d(1,6,3)
        
    def forward(self,x):
        return x

In [25]:
gen_name = "gen"
discr_name = "discr"
if os.path.isfile(gen_name):
    gen = torch.load(gen_name)
else:
    gen = Gen_model()
if os.path.isfile(discr_name):
    discr = torch.load(discr_name)
else:
    discr = Discr_model()
    
optimizer_gen = optim.SGD(gen.parameters(), lr=0.01, momentum=0.99)
optimizer_discr = optim.SGD(discr.parameters(), lr=0.01, momentum=0.99)
criterion = nn.BCELoss()

In [26]:
def train(epoch):
    gen.train()
    discr.train()
    for e in range(epoch):
        avg_loss_gen = 0
        avg_loss_discr = 0
        for batch_id, (data, target) in enumerate(train_loader):
            data = Variable(data[batch_id])
            optimizer_gen.zero_grad()
            optimizer_discr.zero_grad()            
            if (random.getrandbits(1)): # 1 -> real image
                out_discr = discr(data)
                loss_discr = criterion(out_discr,1)
            else: # -> fake image
                out_gen = gen(random.getrandbits(100))
                out_discr = discr(out_gen)
                loss_discr = criterion(out_discr,0)
                loss_gen = criterion(out_discr,1)
                avg_loss_gen += loss_gen.item()
                loss_gen.backward()
                optimizer_gen.step()
            avg_loss_discr += loss_discr.item()
            loss_discr.backward()
            optimizer_discr.step()
            if (batch_id % 1) == 0:
                print("epoch ", e, " [", min(batch_id*batch_size_train,len(train_dataset)), "/", len(train_dataset), "] \naverage loss on gen: ", avg_loss_gen, "\naverage loss on discr: ", avg_loss_discr)
                avg_loss_gen = 0
                avg_loss_discr = 0
        torch.save(gen,gen_name)
        torch.save(discr,discr_name)

In [27]:
train(3)

AttributeError: 'int' object has no attribute 'dim'